In [23]:
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import pandas as pd
from spacy.lang.ru.stop_words import STOP_WORDS

In [2]:
nlp = spacy.load("ru_core_news_md")

#Предобработка текста
def preprocess(text):
    text = text.lower().strip() #приводим к нижнему регистру и избавляеся от лишних пробелов
    text = re.sub(r'<.*?>', '', text)#убираем htlm таги
    text = re.sub(r'[^\w\s]', ' ', text) #убираем пунктицию и символы как ',", # итд
    text = re.sub(r'\s+', ' ', text)# убираем доп пробелы
    return text

#Убирает Шумовые слова как: этом, пока, одном итд
def stopword(string):
    a = [word for word in string.split() if word not in STOP_WORDS]
    return ' '.join(a)

#Приводит слово к изначальному типу
def lemmatizer(string):
    a = nlp(string)
    a = [word.lemma_ for word in a]
    return " ".join(a)

In [3]:
# открываем датасет
df = pd.read_csv("./data/lenta-ru-news.csv", low_memory=False)
df.info()

In [24]:
# удаляем лишние колонки
processed_df = df.drop(['url', 'date', 'topic', 'title'], axis=1)  
processed_df = processed_df.dropna().reset_index()

In [25]:
# объединяем тэги связанные со спортом в одну категорию
processed_df['tags'] = processed_df['tags'].replace(['Футбол','Бокс и ММА', 'Хоккей', 'Английский футбол', 
                                                     'Летние виды', 'Зимние виды', 'Авто', 'Олимпиада', 
                                                     'ЗОЖ и фитнес'] , 'Спорт')

print(processed_df['tags'].value_counts())   

tags
Все               453758
Политика           40716
Общество           35202
Спорт              26236
Украина            22523
                   ...  
Мировой опыт           6
Нацпроекты             6
Вооружение             3
Инновации              1
69-я параллель         1
Name: count, Length: 88, dtype: int64


In [14]:
# Оставляем тэги политика и спорт
processed_df = processed_df.drop(processed_df[(processed_df['tags'] != 'Спорт') & (processed_df['tags'] != 'Политика')].index)

# отбиравем только те тэги, которые встречаются более 1000 раз. В датасете есть много тэгов общей категории. Удалим их тоже
'''processed_df = processed_df.drop(processed_df[(processed_df['tags'] == 'Все')].index)
tag_counts = processed_df['tags'].value_counts()
frequent_tags = tag_counts[tag_counts > 1000].index.tolist()
processed_df = processed_df[processed_df['tags'].isin(frequent_tags)]'''
print(processed_df['tags'].value_counts())   

tags
Политика    40716
Спорт       26236
Name: count, dtype: int64


In [18]:
#создаём словарь с тэгами
categories = {}
for key, value in enumerate(processed_df['tags'].unique()):
    categories[value] = key

processed_df['tag_code'] = processed_df['tags'].map(categories)


In [19]:
print(processed_df.head(10))

         index                                               text      tags  \
448774  448929  В Махачкале в четверг, 27 декабря, был задержа...  Политика   
450202  450363  В Греции неизвестные из автомата Калашникова о...  Политика   
450205  450366  Адвокат оппозиционера Леонида Развозжаева Дмит...  Политика   
450209  450373  Немецкий рэпер Бушидо спровоцировал скандал, о...  Политика   
450219  450386  Нападающий сборной России по хоккею Александр ...     Спорт   
450226  450395  Министерство юстиции РФ отказалось зарегистрир...  Политика   
450261  450434  Генеральный менеджер московского хоккейного ЦС...     Спорт   
450263  450437  Центризбирком РФ предложил заменить названия п...  Политика   
450264  450441  Бывший министр культуры Александр Авдеев назна...  Политика   
450265  450442  Нападающий сборной России по хоккею Николай Же...     Спорт   

        tag_code                                           new_text  
448774         0  махачкала четверг декабрь задержать лидер 

In [20]:
#создаем новый столбец с обработанным текстом
processed_df['new_text'] = processed_df.apply(lambda x: lemmatizer(stopword(preprocess(x['text']))), axis=1)

In [21]:
#создаем новый датасет
processed_df.to_csv('./data/lenta-ru-pl-sp.csv', encoding='utf-8')